##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Getting Started with Gemma 2 and Llamafile

[Gemma](https://ai.google.dev/gemma) is a family of lightweight, state-of-the-art open language models from Google. Built from the same research and technology used to create the Gemini models, Gemma models are text-to-text, decoder-only large language models (LLMs), available in English, with open weights, pre-trained variants, and instruction-tuned variants.

Gemma models are well-suited for a variety of text generation tasks, including question answering, summarization, and reasoning. Their relatively small size makes it possible to deploy them in environments with limited resources such as a laptop, desktop, or your own cloud infrastructure, democratizing access to state-of-the-art AI models and helping foster innovation for everyone.

[Llamafile](https://github.com/Mozilla-Ocho/llamafile) is a tool that simplifies the distribution and execution of open Large Language Models (LLMs) by packaging them into a single-file executable called a "llamafile." By combining llama.cpp with Cosmopolitan Libc, it consolidates the complexity of LLMs into one framework that runs locally on most computers without any installation. The goal is to make open LLMs more accessible to both developers and end users.

In this tutorial, you will learn how to run the Gemma 2 model from Google using Llamafile.

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/Gemma/Using_Gemma_with_Lllamafile.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

## Setup

### Select the Colab runtime
To complete this tutorial, you'll need to have a Colab runtime with sufficient resources to run the Gemma model. In this case, you can use a T4 GPU:

1. In the upper-right of the Colab window, select **▾ (Additional connection options)**.
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **T4 GPU**.

### Gemma setup

**Before we dive into the tutorial, let's get you set up with Gemma:**

1. **Hugging Face Account:**  If you don't already have one, you can create a free Hugging Face account by clicking [here](https://huggingface.co/join).
2. **Gemma Model Access:** Head over to the [Gemma model page](https://huggingface.co/collections/google/gemma-2-release-667d6600fd5220e7b967f315) and accept the usage conditions.
3. **Colab with Gemma Power:**  For this tutorial, you'll need a Colab runtime with enough resources to handle the Gemma 2B model. Choose an appropriate runtime when starting your Colab session.
4. **Hugging Face Token:**  Generate a Hugging Face access (preferably `write` permission) token by clicking [here](https://huggingface.co/settings/tokens). You'll need this token later in the tutorial.

**Once you've completed these steps, you're ready to move on to the next section where we'll set up environment variables in your Colab environment.**


### Configure your HF token

Add your Hugging Face token to the Colab Secrets manager to securely store it.

1. Open your Google Colab notebook and click on the 🔑 Secrets tab in the left panel. <img src="https://storage.googleapis.com/generativeai-downloads/images/secrets.jpg" alt="The Secrets tab is found on the left panel." width=50%>
2. Create a new secret with the name `HF_TOKEN`.
3. Copy/paste your token key into the Value input box of `HF_TOKEN`.
4. Toggle the button on the left to allow notebook access to the secret.


In [ ]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

### Install dependencies
You'll need to install a few Python packages to interact with Hugging Face and run the model.

Run the following cell to install or upgrade it:

In [ ]:
# The huggingface_hub library allows us to download models and other files from Hugging Face.
!pip install --upgrade -q huggingface_hub

### Logging into Hugging Face Hub

Next, you'll have to log into the Hugging Face Hub using your access token. This will allow us to download the Gemma model.

In [ ]:
from huggingface_hub import login

login(os.environ["HF_TOKEN"])

### Downloading the Gemma 2 Model
Once you're logged in, you can download the Gemma 2 model files from Hugging Face. The [Gemma 2 model](https://huggingface.co/google/gemma-2-2b-GGUF) is available in **GGUF** format, which is optimized for use with `llama.cpp` and compatible tools like Llamafile.

In [ ]:
from huggingface_hub import hf_hub_download

# Specify the repository and filename
repo_id = 'google/gemma-2-2b-GGUF'  # Repository containing the GGUF model
filename = '2b_pt_v2.gguf'  # The GGUF model file

# Download the model file to the current directory
hf_hub_download(repo_id=repo_id, filename=filename, local_dir='.')

### Installing Llamafile
Llamafile is a tool that allows you to run Llama and Llama-like models efficiently.


In [ ]:
# Download the latest Llamafile binary (https://github.com/Mozilla-Ocho/llamafile/releases)
!wget -O llamafile https://github.com/Mozilla-Ocho/llamafile/releases/download/0.8.13/llamafile-0.8.13

# Make the binary executable
!chmod +x llamafile

### Run Llamafile
Let's now run Llamafile in server mode, which allows us to interact with it via HTTP requests.

In [ ]:
!nohup ./llamafile --server --nobrowser --port 8081 -ngl 9999 -m 2b_pt_v2.gguf > llamafile.log &

Let's break down the command:

- `nohup`: Runs the command in the background, immune to hangups.
- `./llamafile`: Runs the Llamafile binary.
- `--server`: Starts Llamafile in server mode.
- `--nobrowser`: Prevents Llamafile from opening a browser window.
- `--port`: Specifies the port number to use.
- `-ngl`: Sets the number of GPU layers to offload. Setting it to `9999` offloads as many layers as possible to the GPU.
- `-m`: Specifies the model file to use.
- `> llamafile.log`: Redirects the output to a log file.
- `&`: Runs the process in the background.

**Note:** The `llamafile.log` file will contain logs that can help you troubleshoot any issues.

### Accessing the Llamafile Server
Since you'll be running this in a Colab environment, we need to set up port forwarding to access the Llamafile server.

Use Colab's `eval_js` function to create a proxy URL for port `8081`.

Click on the link provided in the output above to open the Llamafile web interface. From there, you can enter prompts and receive responses from the model.

1. Enter a prompt: In the **input box**, type a question or a statement you would like the model to respond to.  
2. Submit the prompt: Click on **Send** to query the model.  


In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8081)"))

You can now interact with the Gemma 2 model through the Llamafile server.


Congratulations! You've successfully set up the Gemma 2 model using Llamafile in a Colab environment. You can now experiment with the model, generate text, and explore its capabilities.